In [1]:
import json
with open("./board.json", "r") as f:
    board = json.load(f)
print(board)

[{'attack': {'comp': 0, 'vuln': 0}, 'attackerCost': 4, 'defenderCost': 4, 'attackerProb': 0.79, 'defenderProb': 0.69, 'severity': 3, 'risk': 8.82}, {'attack': {'comp': 0, 'vuln': 1}, 'attackerCost': 1, 'defenderCost': 3, 'attackerProb': 0.82, 'defenderProb': 0.97, 'severity': 4, 'risk': 1.18}, {'attack': {'comp': 0, 'vuln': 2}, 'attackerCost': 5, 'defenderCost': 5, 'attackerProb': 0.5, 'defenderProb': 0.77, 'severity': 4, 'risk': 1.84}, {'attack': {'comp': 0, 'vuln': 2}, 'attackerCost': 5, 'defenderCost': 5, 'attackerProb': 0.6, 'defenderProb': 0.77, 'severity': 3, 'risk': 1.84}]


In [2]:
import itertools
def powerset(s):
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1, len(s)+1))

In [8]:
def get_id(move):
    return f"{move['attack']['comp']},{move['attack']['vuln']}"

def get_utility(attackerMoves, defenderMoves):
    defenderLabels = [get_id(m) for m in defenderMoves]
    return sum([
        move["severity"]**2 * move["attackerProb"]
        if get_id(move) not in defenderLabels else
        move["severity"]**2 * move["attackerProb"] * (1-move["defenderProb"])
        for move in attackerMoves
    ])

In [21]:
import numpy as np
all_move_sets = list(powerset(board))
utility_matrix = np.zeros((len(all_move_sets),len(all_move_sets)))
for i, attacker_choice in enumerate(all_move_sets):
    for j, defender_choice in enumerate(all_move_sets):
        utility_matrix[i][j] = get_utility(attacker_choice, defender_choice)

In [25]:
import scipy.io
scipy.io.savemat("../matlab/utility.mat", {
    "utility": utility_matrix,
})